In [5]:
from maze import Maze
from robot import Robot
from take_n_steps import take_n_steps
from print_maze import print_maze
import numpy as np
import sys


# global dictionaries for robot movement and sensing
dir_sensors = {'u': ['l', 'u', 'r'], 'r': ['u', 'r', 'd'],
               'd': ['r', 'd', 'l'], 'l': ['d', 'l', 'u'],
               'up': ['l', 'u', 'r'], 'right': ['u', 'r', 'd'],
               'down': ['r', 'd', 'l'], 'left': ['d', 'l', 'u']}
dir_move = {'u': [0, 1], 'r': [1, 0], 'd': [0, -1], 'l': [-1, 0],
            'up': [0, 1], 'right': [1, 0], 'down': [0, -1], 'left': [-1, 0]}
dir_reverse = {'u': 'd', 'r': 'l', 'd': 'u', 'l': 'r',
               'up': 'd', 'right': 'l', 'down': 'u', 'left': 'r'}

# Create a maze
testmaze = Maze("test_maze_01.txt")

In [41]:
maze_dim = testrobot.maze_dim
heuristic_cost_estimate = np.zeros((maze_dim,maze_dim))
for x in [i for i in range(-maze_dim/2,maze_dim/2+1) if i != 0]:
    for y in [j for j in range(-maze_dim/2,maze_dim/2+1) if j != 0]:
        if x < 0: 
            dx = maze_dim/2 
        else: 
            dx = maze_dim/2 - 1
        if y < 0:
            dy = maze_dim/2
        else:
            dy = maze_dim/2 - 1
        heuristic_cost_estimate[x+dx][y+dy] = (abs(x)+abs(y)) - 2

def a_star(start,goal):
    closedSet = []
    openSet = [start]
    cameFrom = dict()
    
    gScore = {x:maze_dim**2 for x in visited_locations}
    gScore[start] = 0
    fScore = {x:maze_dim**2 for x in visited_locations}
    fScore[start] = heuristic_cost_estimate[start]
    
    def get_min_fscore(locations):
        location_fScore = [fScore[x] for x in locations]
        min_score = min(location_fScore)
        return locations[location_fScore.index(min_score)]
    
    def dist_between(loc1,loc2):
        loc1 = np.array(loc1)
        loc2 = np.array(loc2)
        return sum(abs(loc2-loc1))
    
    while openSet:
        current = get_min_fscore(openSet)
        if current == goal:
            return reconstruct_path(cameFrom,current)
        openSet.remove(current)
        closedSet.append(current)
        
        for next_location in next_locations[current]:
            if next_location in closedSet:
                continue
            
            if next_location not in openSet:
                openSet.append(next_location)
            
            tentative_gScore = gScore[current] + dist_between(current,next_location)
            if tentative_gScore >= gScore[next_location]:
                continue
            
            cameFrom[next_location] = current
            gScore[next_location] = tentative_gScore
            fScore[next_location] = gScore[next_location] + heuristic_cost_estimate[next_location]
            
    print "Failed"

def reconstruct_path(cameFrom,current):
    total_path = [current]
    while current in cameFrom.keys():
        current = cameFrom[current]
        total_path.append(current)
    return total_path

In [62]:
# Intitialize a robot; robot receives info about maze dimensions.
testrobot = Robot(testmaze.dim, (0,0), 'u')

# Set the robot in the start position. Note that robot position
# parameters are independent of the robot itself.
robot_pos = {'location': [0, 0], 'heading': 'u'}

cells = [["   " for x in range(testmaze.dim)] for row in range(testmaze.dim)]     

# Take N steps
hit_goal = False

while testrobot.move_count < 1000 and not hit_goal:
    robot_pos = take_n_steps(1, testmaze, testrobot, robot_pos)

    goal_bounds = [testmaze.dim/2 - 1, testmaze.dim/2]
    if robot_pos['location'][0] in goal_bounds and robot_pos['location'][1] in goal_bounds:
        hit_goal = True

#cells = [["   " for x in range(testmaze.dim)] for row in range(testmaze.dim)]     
#cells[testrobot.current_location[0]][testrobot.current_location[1]] = "XXX"
#print_maze(testmaze,cells)

import get_cells as gc
cells = gc.get_known_cell_values(testrobot,testmaze)
cells[testrobot.current_location[0]][testrobot.current_location[1]] = "XXX"
print testrobot.move_count
#print_maze(testmaze,cells)

195


In [63]:
start = (0,0)
goal = testrobot.current_location
init_next_locations = testrobot.next_locations_table


next_locations = dict()
visited_locations = init_next_locations.keys() 
visited_locations.append(goal)
for location in init_next_locations:
    next_locations[location] = []
    for next_location in init_next_locations[location]:
        for visited_location in visited_locations:
            if next_location[0] == visited_location[0] and next_location[1] == visited_location[1]:
                next_locations[location].append(next_location)

In [64]:
best_path = a_star(start,goal)

In [65]:
import get_cells as gc
cells = gc.get_visited_cells(testrobot,testmaze)
for x in best_path:
    cells[x[0]][x[1]] = " 0 "

print "steps exploring = " + str(testrobot.move_count)
print "best path length = " + str(len(best_path))
print "total score = " + str(testrobot.move_count/30 + len(best_path))
print_maze(testmaze,cells)

steps exploring = 195
best path length = 18
total score = 24
+---+---+---+---+---+---+---+---+---+---+---+---+
| 0   X   0   X   X   X   X   X   X   X   X   X |
+---+---    +---+---+---    +---    +---+---    +
| 0   X   0 | X   X   X   X     | X   X   X   X |
    +---            +---    +---+---+---    +---+
| X   X | X | X | X   X   X   X | X   X   X     |
                +---    +---        +---+---+---+
| X | X   X   X | X   X   X   X | X   X   X     |
    +---+---+---    +---+---+---+---+---        +
| 0   X   0 | X   X   X   X   X   X     | X |   |
+---+---            +---+---        +---        +
| X   X   X   X | X |     0 | X | X   X | X |   |
    +---                                        +
| X   0   0 | X   X |       | X   X | X   X   X |
+---    +---    +---+---    +---        +---    +
| 0   0   X   X |   | 0   0     | X   X | X   X |
    +---+---+---        +---    +---            +
| 0     |     0       0             | X   X |   |
    +---+---        +---    +---    +--

In [4]:
testrobot.wander_setup()

In [85]:
print testrobot.last_location
print next_locations[testrobot.last_location]

(7, 6)
[(6, 6), (5, 6), (7, 7), (7, 5)]


In [17]:
testrobot.wander(1000)

cells = gc.get_known_wander_percentiles(testrobot,testmaze)
cells[testrobot.current_location[0]][testrobot.current_location[1]] = "XXX"
print_maze(testmaze,cells)

+---+---+---+---+---+---+---+---+---+---+---+---+
|        XXX 6.0 4.8 4.3 8.1 7.6 8.4 8.9 4.6 8.3|
+---+---    +---+---+---    +---    +---+---    +
|5.8 4.3 5.5|2.0 2.1 2.0 3.9    |1.6 1.7 2.2 1.6|
    +---            +---    +---+---+---    +---+
|5.9 4.9|3.9|1.2|3.5 7.8 4.1 5.9|3.3 2.2 2.2    |
                +---    +---        +---+---+---+
|5.1|5.7 6.9 5.0|0.0 4.0 2.3 2.6|4.3 5.1 1.4 6.4|
    +---+---+---    +---+---+---+---+---        +
|4.1 1.9 0.0|0.0 0.0 0.0 0.0 3.9 1.7    |7.6|5.8|
+---+---            +---+---        +---        +
|0.0 0.0 0.0 0.0|0.0|       |8.5|1.7 1.8|8.4|2.7|
    +---                                        +
|0.0 0.0 0.0|0.0 0.0|    11.|9.4 1.2|2.0 1.2 1.1|
+---    +---    +---+---    +---        +---    +
|1.4 0.3 0.2 0.2|   |16. 16.    |1.3 1.0|1.4 3.1|
    +---+---+---        +---    +---            +
|2.2    |    12.     15.            |1.4 1.7|   |
    +---+---        +---    +---    +---        +
|2.3     0.2|   |                           |   |
